In [15]:
import tensorflow as tf
# import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow import keras 
import pennylane as qml
import numpy as np

import matplotlib.pyplot as plt

In [16]:
from data_modify import buildpoison,Datapoison

In [17]:
import argparse
import os
import pathlib
parser = argparse.ArgumentParser(description='Reproduce the basic backdoor attack in "Badnets: Identifying vulnerabilities in the machine learning model supply chain".')
parser.add_argument('--dataset', default='MNIST', help='Which dataset to use (MNIST or CIFAR10, default: MNIST)')
parser.add_argument('--data_path', default='./data/', help='Place to load dataset (default: ./dataset/)')
parser.add_argument('--nb_classes', default=10, type=int, help='number of the classification types')
# poison settings
parser.add_argument('--poisoning_rate', type=float, default=0.1, help='poisoning portion (float, range from 0 to 1, default: 0.1)')
parser.add_argument('--trigger_label', type=int, default=1, help='The NO. of trigger label (int, range from 0 to 10, default: 0)')
parser.add_argument('--trigger_path', default="./triggers/trigger_white.png", help='Trigger Path (default: ./triggers/trigger_white.png)')
parser.add_argument('--trigger_size', type=int, default=5, help='Trigger Size (int, default: 5)')

_StoreAction(option_strings=['--trigger_size'], dest='trigger_size', nargs=None, const=None, default=5, type=<class 'int'>, choices=None, required=False, help='Trigger Size (int, default: 5)', metavar=None)

In [18]:
args, unknown = parser.parse_known_args()
X_train, args.nb_classes = buildpoison.build_poisoned_training_set(is_train=True, args=args)

Transform =  Compose(
    ToTensor()
    Normalize(mean=(0.5,), std=(0.5,))
)
Poison 6000 over 60000 samples ( poisoning rate 0.1)
Number of the class = 10
Dataset MNISTPoison
    Number of datapoints: 60000
    Root location: ./data/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )


In [19]:
def one_hot(labels):  
       
    depth = 3**3                       # 10 classes + 6 zeros for padding
    indices = labels.astype(np.int32)    
    one_hot_labels = np.eye(depth)[indices].astype(np.float32) 
    
    return one_hot_labels

# one-hot encoded labels, each label of length cutoff dimension**2
y_train = one_hot(X_train.targets.numpy())
X_train = X_train.data.numpy()

# using only 600 samples for training in this experiment
n_samples = 600
test_samples = 100
X_train, y_train= X_train[:n_samples], y_train[:n_samples]

In [20]:
keras.backend.set_floatx('float32')

model = keras.models.Sequential([
                                 layers.Flatten(input_shape = (28,28)),
                                 layers.Dense(128, activation ="elu"),
                                 layers.Dense(64, activation ="elu"),
                                 layers.Dense(32, activation ="elu"),
                                 layers.Dense(22, activation ="elu") 
                                ])

# More than a million parameters for the classical circuit
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               100480    
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 22)                726       
                                                                 
Total params: 111542 (435.71 KB)
Trainable params: 111542 (435.71 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
def init_layer(x):
    qml.Squeezing(x[0], 0.0, wires=0)
    qml.Squeezing(x[1], 0.0, wires=1)
    qml.Squeezing(x[2], 0.0, wires=2)
    
    qml.Beamsplitter(x[3], x[4], wires=[0,1])
    qml.Beamsplitter(x[5], x[6], wires=[1,2])

    qml.Rotation(x[7], wires=0)
    qml.Rotation(x[8], wires=1)
    qml.Rotation(x[9], wires=2)
   
    qml.Displacement(x[10], 0.0, wires=0)
    qml.Displacement(x[11], 0.0, wires=1)
    qml.Displacement(x[12], 0.0, wires=2)
 
    qml.Kerr(x[13], wires=0)
    qml.Kerr(x[14], wires=1)
    qml.Kerr(x[15], wires=2)

In [22]:
def layer(v):
    qml.Beamsplitter(v[0], v[1], wires=[0,1])
    qml.Beamsplitter(v[2], v[3], wires=[1,2])

    qml.Rotation(v[4], wires=0)
    qml.Rotation(v[5], wires=1)
    qml.Rotation(v[6], wires=2)

    qml.Squeezing(v[7], 0.0, wires=0)
    qml.Squeezing(v[8], 0.0, wires=1)
    qml.Squeezing(v[9], 0.0, wires=2)
    
    qml.Beamsplitter(v[10], v[11], wires=[0,1])
    qml.Beamsplitter(v[12], v[13], wires=[1,2])

    qml.Rotation(v[14], wires=0)
    qml.Rotation(v[15], wires=1)
    qml.Rotation(v[16], wires=2)

    qml.Displacement(v[17], 0.0, wires=0)
    qml.Displacement(v[18], 0.0, wires=1)
    qml.Displacement(v[19], 0.0, wires=2)

    qml.Kerr(v[20], wires=0)
    qml.Kerr(v[21], wires=1)
    qml.Kerr(v[22], wires=2)

In [23]:
def init_weights(layers, modes, active_sd=0.0001, passive_sd=0.1):
    
    M = 4+3  # Number of interferometer parameters: 2 2-parameter beamsplitters + 3 rotations

    int1_weights = tf.random.normal(shape=[layers, M], stddev=passive_sd)
    s_weights = tf.random.normal(shape=[layers, modes], stddev=active_sd)
    int2_weights = tf.random.normal(shape=[layers, M], stddev=passive_sd)
    dr_weights = tf.random.normal(shape=[layers, modes], stddev=active_sd)
    k_weights = tf.random.normal(shape=[layers, modes], stddev=active_sd)

    weights = tf.concat([int1_weights, s_weights, int2_weights, dr_weights, k_weights], axis=1)
    weights = tf.Variable(weights)

    return weights

In [28]:
num_modes = 3
num_basis = 3

dev = qml.device("strawberryfields.fock", wires=num_modes, cutoff_dim=num_basis) 

@qml.qnode(dev, interface="tf")
def quantum_nn(inputs, var):
    init_layer(inputs)            # Encode input x into quantum state

    for v in var:                 # iterative quantum layers
        layer(v)

    return qml.probs(wires=[0, 1, 2])  # Measurement 

In [29]:
num_layers = 4

# initialize weights for quantum layers
weights = init_weights(num_layers, num_modes)

# Convert the quantum layer to a Keras layer
shape_tup = weights.shape
weight_shapes = {'var': shape_tup}
qlayer = qml.qnn.KerasLayer(quantum_nn, weight_shapes, output_dim = 4)

# Add to the classical sequential model
model.add(qlayer)

In [30]:
opt = keras.optimizers.SGD(lr = 0.03)
model.compile(opt, loss = 'categorical_crossentropy', metrics =['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               100480    
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 22)                726       
                                                                 
 keras_layer_1 (KerasLayer)  (None, 4)                 92        
                                                                 
 keras_layer_2 (KerasLayer)  (None, 4)                

In [31]:
hybrid = model.fit(X_train, 
                   y_train,
                   epochs = 100,
                   batch_size = 64,
                   shuffle = True, 
                   validation_data = (X_train, y_train))

Epoch 1/100


ImportError: Exception encountered when calling layer 'keras_layer_1' (type KerasLayer).

cannot import name 'should_record' from 'tensorflow.python.eager.tape' (c:\Users\32827\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\eager\tape.py)

Call arguments received by layer 'keras_layer_1' (type KerasLayer):
  • inputs=tf.Tensor(shape=(64, 22), dtype=float32)